## Tensorflow Serving Version 1.13 Example

Train and export a simple Softmax Regression TensorFlow model.
The model is from the TensorFlow "MNIST For ML Beginner" tutorial. This program
simply follows all its training instructions, and uses TensorFlow SavedModel to
export the trained model with proper signatures that can be loaded by standard
tensorflow_model_server.
<br>
Usage: mnist_saved_model.py [--training_iteration=x] [--model_version=y] export_dir

In [1]:
from __future__ import print_function

import os
import sys
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.contrib.session_bundle import exporter

In [2]:
export_path_base = "./model"
training_iterations = 1000
model_version = 1

In [3]:
# training data
train_height = np.array([1.60, 1.62, 1.68, 1.69, 1.70, 1.70, 1.71, 1.72, 1.80, 1.76, 1.77, 1.78, 1.77, 1.78, 1.79, 1.80, 1.81, 1.82, 1.83, 1.94])
train_weight = np.array([60.5, 58.8, 62.7, 62.8, 63.1, 58.9, 61.5, 65.8, 70.0, 65.8, 66.8, 68.0, 66.7, 60.8, 72.4, 70.5, 72.8, 75.8, 74.6, 83.2])

In [4]:
sess = tf.InteractiveSession()
 
x = tf.placeholder('float', shape=[None, 3])
y_ = tf.placeholder('float', shape=[None, 1])

w = tf.get_variable('w', shape = [3,1], initializer = tf.truncated_normal_initializer)
b = tf.get_variable('b', shape = [1], initializer = tf.zeros_initializer)
 
sess.run(tf.global_variables_initializer())
 
y = tf.matmul(x, w) + b
 
ms_loss = tf.reduce_mean((y - y_)**2)
 
train_step = tf.train.GradientDescentOptimizer(0.005).minimize(ms_loss)
 
train_x = np.random.randn(1000, 3)
# let the model learn the equation of y = x1 * 1 + x2 * 2 + x3 * 3
train_y = np.sum(train_x * np.array([1,2,3]) + np.random.randn(1000, 3) / 100, axis = 1).reshape(-1, 1)
 
train_loss = []
 
for it in range(training_iterations):
    loss, _ = sess.run([ms_loss, train_step], feed_dict={x: train_x, y_: train_y})
    train_loss.append(loss)
    
print('Training error %g' % loss)
print('Done training!')


export_path = os.path.join(
  tf.compat.as_bytes(export_path_base),
  tf.compat.as_bytes(str(model_version)))

print('Exporting trained model to', export_path)
builder = tf.saved_model.builder.SavedModelBuilder(export_path)
 
tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
tensor_info_y = tf.saved_model.utils.build_tensor_info(y)
 
prediction_signature = (
  tf.saved_model.signature_def_utils.build_signature_def(
  inputs={'input': tensor_info_x},
  outputs={'output': tensor_info_y},
  method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
)
 
legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
 
builder.add_meta_graph_and_variables(
  sess, [tf.saved_model.tag_constants.SERVING],
  signature_def_map={
  'prediction':
  prediction_signature,
  },
  legacy_init_op=legacy_init_op)
 
builder.save()
 
print('Done exporting!')

Instructions for updating:
Colocations handled automatically by placer.
Training error 0.000287986
Done training!
Exporting trained model to b'./model/1'
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./model/1/saved_model.pb
Done exporting!


### Start Tensorflow Serving (don't specify model version)

docker run -p 8501:8501 --mount type=bind,source=/home/bai/PycharmProjects/tensorflow_for_enginner/11_tf_serving/model/1,target=/models/my_model -e MODEL_NAME=my_model -t tensorflow/serving